# 2D DDW
## Pedicting Crop Type from data based in India



In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### **Model Outline(stuff yall can follow):**
> **Warning:** DO NOT MANIPULATE THE CSV FILES DIRECTLY. Any manipulation will be done by loading the data into a variable in your code. CSV FILES SHOULD BE UNTOUCHED.


#### Step 1 Data Preprocessing:

**CSV Files**
```bash
├── Fertilizer.csv
├── Final_Dataset_after_temperature.csv
```
- Display both files using pandas and take a look at the columns. You need to find a way to combine this datasets and remove some of the unnecessary columns(ex. Production column in final dataset.)
- After which, recall what i said about data that are not numbers, in this case states. Look up panda dummy variables.
- Normalize the data. Can use the one in problems set.
- Use charts to visualize, summarize and understand your data. Whatever stuff you guys can think of that will help someone to understand the data better(I'll do this~Gangesh)
  
#### Step 2 Training and Validation:
- Follow the initial steps exactly as in cohort 9 to train your first model
- evaluate the model using an appropriate metric. Indicate all data of the trained model together with the error through your chosen error function. Visulize the trained model with maybe types of graph
- Take steps to improve your model. Try to search online. One way is to vary the iterations and learning rate. Train a few times, then choose the model with the best accuracy.

#### Step 3 Prediction:
Now that you have your trained model, you need to find a way to predict the crop type that gives the highest yield. Recall that the above model predicts highest yield for any given crop type. So what you would do is create a function where the user types in everything except for crop type, and you would go on to predict yield for every crop type with the inputted variables. After which you will choose the crop type with the highest yield and return it 
